In [18]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
import itertools
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [19]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Downloads\Compressed\chromedriver_win32\chromedriver.exe',options=chrome_options)

In [20]:
url='https://www.opuskinetic.com/trainings/'

driver.get(url)
time.sleep(5)

crads=[]
sef=True
while sef:
    soup=bs(driver.page_source,'lxml')
    car=soup.find('table',{'class':'table-event-list table-striped table-responsive table-bordered'}).find('tbody').find_all('tr',{'class':re.compile('tribe-mini-calendar-event.+')})
    np=soup.find('div',{'id':'tribe-events-footer'}).find('li',{'class':'tribe-events-nav-right'}).find('a')['href']
    if np=='undefined':
        sef=False
    else:
        try:
            driver.get(np)
        except:
            pass
    crads.append(car)
cards=(list(itertools.chain.from_iterable(crads)))
adde=[]
for i in cards:
    ritle=' '.join(i.find('td',{'class':'td-title'}).text.split())
    link=i.find('a')['href']
    lo=' '.join(i.find('td',{'class':'td-location'}).text.split())
    dat=' '.join(i.find('td',{'class':'td-date'}).text.split()).split()
    dur=' '.join(i.find('td',{'class':'td-duration'}).text.split()).split()[0]
    dy=dat[0]+'-'+str(int(dat[0])+int(dur))
    adde.append([ritle,link,lo,dat,dur,dy])

In [25]:
opus=[]
for k in adde:
    link=k[1]
    loc=k[2]
    try:
        driver.get(link)
    except:
        pass
    soupn=bs(driver.page_source,'lxml')
    title=' '.join(soupn.find('h1',{'class':'tribe-events-single-event-title'}).text.split())
    dat_dur=' '.join(soupn.find('div',{'class':'tribe-events-schedule tribe-clearfix'}).find('h2').text.split())
    if not re.search('\d{4}',dat_dur):
        yr=str(date.today().year)
        dat_dur=dat_dur+' '+yr
    dat=dat_dur.replace('–','-').replace('|','').replace('to','-').replace('TO','-').strip().replace('c-ber','ctober').replace('C-BER','CTOBER')
    if '(' in dat:
        dat=dat.split('(')[0].strip()
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    else:
        st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
   
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
    
    event_info=' '.join(soupn.find('div',{'class':'tribe-events-single-event-description tribe-events-content'}).find('p').text.split())
    
    #     #**************
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location+', University of Michigan'))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        locale=locale+', University of Michigan'
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")
    time_=''
    ticket_list=''
    org_name='Opus Kinetic Pte Ltd'
    org_web='https://www.opuskinetic.com/'
    org_pro='We are the premier provider of knowledge through people empowerment. We believe people are why organisations are successful, and giving people the knowledge to perform well at their job will enable organisations to succeed. That is why we pride ourselves on offering acclaimed training courses that are well researched and follow the latest industry trends.'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
    Speakerlist='' 
    event_web=link
    
    mail_=json.dumps(['info@opuskinetic.com'])
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-'or 'F2F' in loc or 'lassroom' in loc or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        if ',' in loc:
            city=loc.split(',')[-2].strip()
            country=loc.split(',')[-1].strip()
        else:
            city=loc
            country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)


    opus.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])

    
  

Singapore ;country didn't go through
Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A16463+2188387]
	Ordinal0 [0x009AE461+1762401]
	Ordinal0 [0x008C3D78+802168]
	Ordinal0 [0x008B7210+750096]
	Ordinal0 [0x008B675A+747354]
	Ordinal0 [0x008B5D3F+744767]
	Ordinal0 [0x008B4C28+740392]
	Ordinal0 [0x008B5228+741928]
	Ordinal0 [0x008BEF2F+782127]
	Ordinal0 [0x008C9FBB+827323]
	Ordinal0 [0x008CD310+840464]
	Ordinal0 [0x008B54F6+742646]
	Ordinal0 [0x008C9BF3+826355]
	Ordinal0 [0x0091CF6D+1167213]
	Ordinal0 [0x0090C5F6+1099254]
	Ordinal0 [0x008E6BE0+945120]
	Ordinal0 [0x008E7AD6+948950]
	GetHandleVerifier [0x00CB71F2+2712546]
	GetHandleVerifier [0x00CA886D+2652765]
	GetHandleVerifier [0x00AA002A+520730]
	GetHandleVerifier [0x00A9EE06+516086]
	Ordinal0 [0x009B468B+1787531]
	Ordinal0 [0x009B8E88+1805960]
	Ordinal0 [0x009B8F75+1806197]
	Ordinal0 [0x009C1DF1+1842673]


In [26]:
len(opus)

110

In [27]:
opus

[['https://www.opuskinetic.com/training/masterclass-on-understanding-drilling-contracts-2/',
  'Understanding Drilling Contracts',
  '2022-07-04',
  '2022-07-07',
  '',
  'This is a 4-day drilling contracts training. The ‘Upstream’ or Exploration and Production (E&P) segment of the International Oil and Gas Industry, is in all probability, the riskiest in terms of capital intensity, long payback periods; technological dependence and a host of legal/environmental compliance requirements. It is commonplace for entities seeking hydrocarbon reserves to offset this risk exposure through collective entities such as Joint Ventures. This drilling contracts training course has been specifically designed to provide the delegate with a holistic understanding not only of drilling contracts, but to understand them within the wider context of the larger governing framework of which they are a part in terms of hydrocarbon discovery and extraction. The course is an immersive experience and the focus i

In [29]:
opus_dict=opus


opus_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=opus_dict)
opus_df.to_csv("opus.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
